In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import GroupKFold
import zipfile
from sklearn.linear_model import Ridge,LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
import numpy as np
import lightgbm as lgb


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/CommonLit/commont-lit-train.csv")

gkf = GroupKFold(n_splits=4)
for i, (_, val_index) in enumerate(gkf.split(train, groups=train["prompt_id"])):
    train.loc[val_index, "fold"] = i


In [ ]:
albert = pd.read_csv("/content/drive/MyDrive/CommonLit/albert-base-v2/train-predictions.csv")
electra = pd.read_csv("/content/drive/MyDrive/CommonLit/electra-base-discriminator/train-predictions.csv")
roberta = pd.read_csv("/content/drive/MyDrive/CommonLit/roberta-base-squad2/train-predictions.csv")
deberta = pd.read_csv("/content/drive/MyDrive/CommonLit/deberta-v3-base/train-predictions.csv")
xlm = pd.read_csv("/content/drive/MyDrive/CommonLit/xlm-roberta-base/train-predictions.csv")
funnel =  pd.read_csv("/content/drive/MyDrive/CommonLit/funnel-medium-base/train-predictions.csv")
mpnet = pd.read_csv("/content/drive/MyDrive/CommonLit/all-mpnet-base-v2/train-predictions.csv")
distilbert_base = pd.read_csv("/content/drive/MyDrive/CommonLit/distilbert-base-uncased/train-predictions.csv")
distilbert_base_sst = pd.read_csv("/content/drive/MyDrive/CommonLit/distilbert-base-uncased-finetuned-sst-2-english/train-predictions.csv")
bert_base =  pd.read_csv("/content/drive/MyDrive/CommonLit/bert-base-uncased/train-predictions.csv")
deberta_v3_large = pd.read_csv("/content/drive/MyDrive/CommonLit/deberta-v3-large/train-predictions_1.csv")
xlm_roberta_large = pd.read_csv("/content/drive/MyDrive/CommonLit/xlm-roberta-large/train-predictions_1.csv")
roberta_large = pd.read_csv("/content/drive/MyDrive/CommonLit/roberta-large/train-predictions_1.csv")
bert_large_cased = pd.read_csv("/content/drive/MyDrive/CommonLit/bert-large-cased/train-predictions_1.csv")

albert_pred = albert[['content_multi_pred','wording_multi_pred']].copy()
electra_pred = electra[['content_multi_pred','wording_multi_pred']].copy()
roberta_pred = roberta[['content_multi_pred','wording_multi_pred']].copy()
deberta_pred = deberta[['content_multi_pred','wording_multi_pred']].copy()
xlm_pred = xlm[['content_multi_pred','wording_multi_pred']].copy()
funnel_pred = funnel[['content_multi_pred','wording_multi_pred']].copy()
mpnet_pred = mpnet[['content_multi_pred','wording_multi_pred']].copy()
distilbert_base_pred = distilbert_base[['content_multi_pred','wording_multi_pred']].copy()
distilbert_base_sst_pred = distilbert_base_sst[['content_multi_pred','wording_multi_pred']].copy()
bert_base_pred = bert_base[['content_multi_pred','wording_multi_pred']].copy()
deberta_v3_large_pred = deberta_v3_large[['content_multi_pred','wording_multi_pred']].copy()
xlm_roberta_large_pred = xlm_roberta_large[['content_multi_pred','wording_multi_pred']].copy()
roberta_large_pred = roberta_large[['content_multi_pred','wording_multi_pred']].copy()
bert_large_cased_pred = bert_large_cased[['content_multi_pred','wording_multi_pred']].copy()

albert_pred = albert_pred.rename(columns={"content_multi_pred":"albert_content","wording_multi_pred":"albert_wording"})
electra_pred = electra_pred.rename(columns={"content_multi_pred":"electra_content","wording_multi_pred":"electra_wording"})
roberta_pred = roberta_pred.rename(columns={"content_multi_pred":"roberta_content","wording_multi_pred":"roberta_wording"})
deberta_pred = deberta_pred.rename(columns={"content_multi_pred":"deberta_content","wording_multi_pred":"deberta_wording"})
xlm_pred = xlm_pred.rename(columns={"content_multi_pred":"xlm_content","wording_multi_pred":"xlm_wording"})
funnel_pred = funnel_pred.rename(columns={"content_multi_pred":"funnel_content","wording_multi_pred":"funnel_wording"})
mpnet_pred = mpnet_pred.rename(columns={"content_multi_pred":"mpnet_content","wording_multi_pred":"mpnet_wording"})
distilbert_base_pred = distilbert_base_pred.rename(columns={"content_multi_pred":"distilbert_content","wording_multi_pred":"distilbert_wording"})
distilbert_base_sst_pred = distilbert_base_sst_pred.rename(columns={"content_multi_pred":"distilbert_sst_content","wording_multi_pred":"distilbert_sst_wording"})
bert_base_pred = bert_base_pred.rename(columns={"content_multi_pred":"bert_content","wording_multi_pred":"bert_wording"})
deberta_v3_large_pred = deberta_v3_large_pred.rename(columns={"content_multi_pred":"deberta_v3_large_content","wording_multi_pred":"deberta_v3_large_wording"})
xlm_roberta_large_pred = xlm_roberta_large_pred.rename(columns={"content_multi_pred":"xlm_roberta_large_content","wording_multi_pred":"xlm_roberta_large_wording"})
roberta_large_pred = roberta_large_pred.rename(columns={"content_multi_pred":"roberta_large_content","wording_multi_pred":"roberta_large_wording"})
bert_large_cased_pred = bert_large_cased_pred.rename(columns={"content_multi_pred":"bert_large_cased_content","wording_multi_pred":"bert_large_cased_wording"})

In [ ]:
all_preds = pd.concat([train[['content','wording','fold']].copy(),
                       xlm_pred,funnel_pred,mpnet_pred,deberta_v3_large_pred],axis=1)
all_preds

,content,wording,fold,xlm_content,xlm_wording,funnel_content,funnel_wording,mpnet_content,mpnet_wording,deberta_v3_large_content,deberta_v3_large_wording
0,0.205683,0.380538,3.0,0.048096,0.604004,-0.164185,0.444824,0.245850,1.056641,0.060547,0.817871
1,-0.548304,0.506755,2.0,-0.484619,-0.054352,-0.560059,-0.255615,-0.372070,0.034302,-0.770508,-0.442871
2,3.128928,4.231226,1.0,2.257812,1.947266,1.828125,1.658203,2.093750,2.015625,2.558594,2.748047
3,-0.210614,-0.471415,1.0,-1.192383,-1.055664,-0.964844,-0.855957,-0.900879,-0.801758,-0.699219,-0.434326
4,3.272894,3.219757,3.0,2.115234,1.890625,2.437500,2.500000,2.095703,2.023438,1.975586,2.103516
...,...,...,...,...,...,...,...,...,...,...,...
7160,0.205683,0.380538,2.0,-0.023285,-0.028091,-0.281982,-0.092346,-0.187500,0.138306,-0.028366,0.094543
7161,-0.308448,0.048171,1.0,-0.574707,-0.354004,-0.511230,-0.302979,-0.427734,-0.218628,-0.197021,-0.052368
7162,-1.408180,-0.493603,1.0,-0.751953,-0.255859,-1.231445,-1.173828,-0.592773,-0.391602,-0.518066,-0.248413
7163,-0.393310,0.627128,0.0,0.258301,0.458008,0.062286,0.427246,-0.195557,0.263184,-0.023483,0.413086


###Regular Ensemble

In [ ]:
drop_columns = ['content','wording','fold']
rmses = []

for target in ['content','wording']:
    preds = []
    trues = []

    for fold in range(4):
        X_eval_cv = all_preds[all_preds["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = all_preds[all_preds["fold"] == fold][target]
        X_eval_cv = X_eval_cv.filter(regex=(f'_{target}'))
        pred = X_eval_cv.apply(np.mean,axis=1)

        trues.extend(y_eval_cv)
        preds.extend(pred)


    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.45367627071428196
wording_rmse : 0.6334629437465463
mcrmse : 0.5435696072304141


###Regular with LGBM

In [ ]:
targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text",
                "prompt_question", "prompt_title",
                "prompt_text","prompt_length",
                "avg_word_length","semicolon_count","neg","neu","pos","compound",
                "exclamation_count","question_count","punctuation_sum","neg_prompt","neu_prompt","pos_prompt",
                "compound_prompt","flesch_reading_ease","flesch_kincaid_grade","gunning_fog","automated_readability_index",
                "coleman_liau_index","linsear_write_formula","dale_chall_readability_score","text_standard","spache_readability",
                "mcalpine_eflaw",
               ] + targets


model_dict = {}

for target in targets:
    models = []

    for fold in range(4):

        X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns)
        y_train_cv = train[train["fold"] != fold][target]

        mean_pred = all_preds[all_preds['fold']!=fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_content'))
        mean_pred = mean_pred.apply(np.mean,axis=1)
        X_train_cv['mean_pred_content'] = mean_pred

        mean_pred = all_preds[all_preds['fold']!=fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_wording'))
        mean_pred = mean_pred.apply(np.mean,axis=1)
        X_train_cv['mean_pred_wording'] = mean_pred

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        mean_pred = all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_content'))
        mean_pred = mean_pred.apply(np.mean,axis=1)
        X_eval_cv['mean_pred_content'] = mean_pred

        mean_pred = all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_wording'))
        mean_pred = mean_pred.apply(np.mean,axis=1)
        X_eval_cv['mean_pred_wording'] = mean_pred


        dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
        dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)

        params = {
                  'boosting_type': 'gbdt',
                  'random_state': 42,
                  'objective': 'regression',
                  'metric': 'rmse',
                  'learning_rate': 0.048,
                  'lambda_l1': 0.0,
                  'lambda_l2': 0.011
                  }

        evaluation_results = {}
        model = lgb.train(params,
                          num_boost_round=10000,
                            #categorical_feature = categorical_features,
                          valid_names=['train', 'valid'],
                          train_set=dtrain,
                          valid_sets=dval,
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=30, verbose=False),
                            #    lgb.log_evaluation(100),
                            #   lgb.callback.record_evaluation(evaluation_results)
                            ],
                          )
        models.append(model)

    model_dict[target] = models

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001317 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3004
[LightGBM] [Info] Number of data points in the train set: 5108, number of used features: 21
[LightGBM] [Info] Start training from score 0.017606
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001312 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2926
[LightGBM] [Info] Number of data points in the train set: 5156, number of used features: 21
[LightGBM] [Info] Start training from score -0.039959
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.158427 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2927
[LightGBM] [Info] Number of data points in the t

In [ ]:
# cv
rmses = []

for target in targets:
    models = model_dict[target]

    preds = []
    trues = []

    for fold, model in enumerate(models):

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        mean_pred = all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_content'))
        mean_pred = mean_pred.apply(np.mean,axis=1)
        X_eval_cv['mean_pred_content'] = mean_pred

        mean_pred = all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_wording'))
        mean_pred = mean_pred.apply(np.mean,axis=1)
        X_eval_cv['mean_pred_wording'] = mean_pred


        pred = model.predict(X_eval_cv)

        trues.extend(y_eval_cv)
        preds.extend(pred)

    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.4316029469934855
wording_rmse : 0.5660460604544022
mcrmse : 0.4988245037239438


###Bayesian Optimization

In [ ]:
pbounds = {
   'w0': (0.01, 0.99),
   'w1': (0.01, 0.99),
   'w2': (0.01, 0.99),
   'w3': (0.01, 0.99),
   'w4': (0.01, 0.99),
   'w5': (0.01, 0.99),
   'w6': (0.01, 0.99),
}

content_pred = all_preds.filter(regex=(f'_content'))
pred_0 = content_pred[content_pred.columns[0]]
pred_1 = content_pred[content_pred.columns[1]]
pred_2 = content_pred[content_pred.columns[2]]
pred_3 = content_pred[content_pred.columns[3]]
pred_4 = content_pred[content_pred.columns[4]]
pred_5 = content_pred[content_pred.columns[5]]
pred_6 = content_pred[content_pred.columns[6]]
y_actual = all_preds['content']

def objective(w0, w1, w2, w3, w4, w5, w6):
   weighted_avg_pred_logits = [w0*x + w1*y + w2*z + w3*t + w4*a +w5*b +w6*c for x, y, z, t,a,b,c in zip(pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6)]
#    weighted_avg_pred = np.argmax(weighted_avg_pred_logits, axis=1)
   return 1-mean_squared_error(y_actual,weighted_avg_pred_logits,squared=False)

In [ ]:
optimizer = BayesianOptimization(
   f=objective,
   pbounds=pbounds,
   verbose=2,
   random_state=42,
   allow_duplicate_points=True
)

###Nelder-Mead

In [ ]:
import random
from numpy.random import rand
from scipy.optimize import minimize

In [ ]:
drop_columns = ['content','wording','fold']
rmses = []

for target in ['content','wording']:
    preds = []
    trues = []

    for fold in range(4):
        X_eval_cv = all_preds[all_preds["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = all_preds[all_preds["fold"] == fold][target]
        X_eval_cv = X_eval_cv.filter(regex=(f'_{target}'))

        pred_0 = X_eval_cv[X_eval_cv.columns[0]]
        pred_1 = X_eval_cv[X_eval_cv.columns[1]]
        pred_2 = X_eval_cv[X_eval_cv.columns[2]]
        pred_3 = X_eval_cv[X_eval_cv.columns[3]]
        pred_4 = X_eval_cv[X_eval_cv.columns[4]]
        # pred_5 = X_eval_cv[X_eval_cv.columns[5]]
        # pred_6 = X_eval_cv[X_eval_cv.columns[6]]

        def objective(x):
            w0, w1, w2, w3, w4 = x
            weighted_avg_pred_logits = [w0*x + w1*y + w2*z + w3*t + w4*a for x, y, z, t,a, in zip(pred_0, pred_1, pred_2, pred_3, pred_4)]
            return mean_squared_error(y_eval_cv,weighted_avg_pred_logits,squared=False)


        pt = rand(5)
        result = minimize(objective, pt, method='nelder-mead')
        solution = result['x']

        for i,col in enumerate(X_eval_cv):
            X_eval_cv[col] = X_eval_cv[col].multiply(solution[i])

        pred = X_eval_cv.apply(np.sum,axis=1)
        trues.extend(y_eval_cv)
        preds.extend(pred)


    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.4419184953066903
wording_rmse : 0.5942609097578907
mcrmse : 0.5180897025322905


####LGBM+Nelder-mead

In [ ]:
targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text",
                "prompt_question", "prompt_title",
                "prompt_text","prompt_length",
                "avg_word_length","semicolon_count","neg","neu","pos","compound",
                "exclamation_count","question_count","punctuation_sum","neg_prompt","neu_prompt","pos_prompt",
                "compound_prompt","flesch_reading_ease","flesch_kincaid_grade","gunning_fog","automated_readability_index",
                "coleman_liau_index","linsear_write_formula","dale_chall_readability_score","text_standard","spache_readability",
                "mcalpine_eflaw"
               ] + targets

model_dict = {}

for target in targets:
    models = []

    for fold in range(4):

        X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns)
        y_train_cv = train[train["fold"] != fold][target]

        ####Content
        mean_pred = all_preds[all_preds['fold']!=fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=('_content'))

        pred_0 = mean_pred[mean_pred.columns[0]]
        pred_1 = mean_pred[mean_pred.columns[1]]
        pred_2 = mean_pred[mean_pred.columns[2]]
        pred_3 = mean_pred[mean_pred.columns[3]]
        pred_4 = mean_pred[mean_pred.columns[4]]


        def objective(x):
            w0, w1, w2, w3, w4= x
            weighted_avg_pred_logits = [w0*x + w1*y + w2*z + w3*t + w4*a for x, y, z, t,a in zip(pred_0, pred_1, pred_2, pred_3, pred_4)]
            return mean_squared_error(y_train_cv,weighted_avg_pred_logits,squared=False)


        pt = rand(5)
        result = minimize(objective, pt, method='nelder-mead')
        solution = result['x']

        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(solution[i])

        X_train_cv[f'nelder_mead_pred_content'] =  mean_pred.apply(np.sum,axis=1)

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        mean_pred = all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_content'))

        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(solution[i])

        X_eval_cv[f'nelder_mead_pred_content'] =  mean_pred.apply(np.sum,axis=1)


        ##################################################################
        #### Wording

        mean_pred = all_preds[all_preds['fold']!=fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=('_wording'))

        pred_0 = mean_pred[mean_pred.columns[0]]
        pred_1 = mean_pred[mean_pred.columns[1]]
        pred_2 = mean_pred[mean_pred.columns[2]]
        pred_3 = mean_pred[mean_pred.columns[3]]
        pred_4 = mean_pred[mean_pred.columns[4]]

        def objective(x):
            w0, w1, w2, w3, w4 = x
            weighted_avg_pred_logits = [w0*x + w1*y + w2*z + w3*t + w4*a for x, y, z, t,a in zip(pred_0, pred_1, pred_2, pred_3, pred_4)]
            return mean_squared_error(y_train_cv,weighted_avg_pred_logits,squared=False)


        pt = rand(5)
        result = minimize(objective, pt, method='nelder-mead')
        solution = result['x']

        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(solution[i])

        X_train_cv[f'nelder_mead_pred_wording'] =  mean_pred.apply(np.sum,axis=1)

        # X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        # y_eval_cv = train[train["fold"] == fold][target]

        mean_pred = all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_wording'))

        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(solution[i])

        X_eval_cv[f'nelder_mead_pred_wording'] =  mean_pred.apply(np.sum,axis=1)



        dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
        dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)

        params = {
                  'boosting_type': 'gbdt',
                  'random_state': 42,
                  'objective': 'regression',
                  'metric': 'rmse',
                  'learning_rate': 0.048,
                  'lambda_l1': 0.0,
                  'lambda_l2': 0.011
                  }

        evaluation_results = {}
        model = lgb.train(params,
                          num_boost_round=10000,
                            #categorical_feature = categorical_features,
                          valid_names=['train', 'valid'],
                          train_set=dtrain,
                          valid_sets=dval,
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=30, verbose=True),
                               lgb.log_evaluation(100),
                              lgb.callback.record_evaluation(evaluation_results)
                            ],
                          )
        models.append(model)

    model_dict[target] = models

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001274 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3004
[LightGBM] [Info] Number of data points in the train set: 5108, number of used features: 21
[LightGBM] [Info] Start training from score 0.017606
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[59]	train's rmse: 0.396699
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001281 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2926
[LightGBM] [Info] Number of data points in the train set: 5156, number of used features: 21
[LightGBM] [Info] Start training from score -0.039959
Training until validation scores don't improve for 30 rounds
[100]	train's rmse: 0.470891
Early stopping, best iteration is:
[126]	train's rmse: 0.4705
[LightGBM] [Warning] Auto-choosing 

In [ ]:
# cv
rmses = []

for target in targets:
    models = model_dict[target]

    preds = []
    trues = []

    for fold, model in enumerate(models):

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        #### Content
        mean_pred = all_preds[all_preds['fold']!=fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_content'))

        pred_0 = mean_pred[mean_pred.columns[0]]
        pred_1 = mean_pred[mean_pred.columns[1]]
        pred_2 = mean_pred[mean_pred.columns[2]]
        pred_3 = mean_pred[mean_pred.columns[3]]
        pred_4 = mean_pred[mean_pred.columns[4]]


        def objective(x):
            w0, w1, w2, w3, w4 = x
            weighted_avg_pred_logits = [w0*x + w1*y + w2*z + w3*t + w4*a  for x, y, z, t,a in zip(pred_0, pred_1, pred_2, pred_3, pred_4)]
            return mean_squared_error(all_preds[all_preds['fold']!=fold][target],weighted_avg_pred_logits,squared=False)

        pt = rand(5)
        result = minimize(objective, pt, method='nelder-mead')
        solution = result['x']

        mean_pred = all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_content'))
        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(solution[i])

        X_eval_cv['nelder_mead_pred_content'] =  mean_pred.apply(np.sum,axis=1)


        ##################################################################
        ####Wording

        mean_pred = all_preds[all_preds['fold']!=fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_wording'))

        pred_0 = mean_pred[mean_pred.columns[0]]
        pred_1 = mean_pred[mean_pred.columns[1]]
        pred_2 = mean_pred[mean_pred.columns[2]]
        pred_3 = mean_pred[mean_pred.columns[3]]
        pred_4 = mean_pred[mean_pred.columns[4]]

        def objective(x):
            w0, w1, w2, w3, w4 = x
            weighted_avg_pred_logits = [w0*x + w1*y + w2*z + w3*t + w4*a  for x, y, z, t,a in zip(pred_0, pred_1, pred_2, pred_3, pred_4)]
            return mean_squared_error(all_preds[all_preds['fold']!=fold][target],weighted_avg_pred_logits,squared=False)

        pt = rand(5)
        result = minimize(objective, pt, method='nelder-mead')
        solution = result['x']

        mean_pred = all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_wording'))
        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(solution[i])

        X_eval_cv['nelder_mead_pred_wording'] =  mean_pred.apply(np.sum,axis=1)





        pred = model.predict(X_eval_cv)

        trues.extend(y_eval_cv)
        preds.extend(pred)

    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.4324876736391451
wording_rmse : 0.5824039631737561
mcrmse : 0.5074458184064505


###Dual Annealing

In [ ]:
from scipy.optimize import dual_annealing

In [ ]:
drop_columns = ['content','wording','fold']
rmses = []

for target in ['content','wording']:
    preds = []
    trues = []

    for fold in range(4):
        X_eval_cv = all_preds[all_preds["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = all_preds[all_preds["fold"] == fold][target]
        X_eval_cv = X_eval_cv.filter(regex=(f'_{target}'))

        pred_0 = X_eval_cv[X_eval_cv.columns[0]]
        pred_1 = X_eval_cv[X_eval_cv.columns[1]]
        pred_2 = X_eval_cv[X_eval_cv.columns[2]]
        pred_3 = X_eval_cv[X_eval_cv.columns[3]]
        pred_4 = X_eval_cv[X_eval_cv.columns[4]]
        pred_5 = X_eval_cv[X_eval_cv.columns[5]]
        pred_6 = X_eval_cv[X_eval_cv.columns[6]]

        def objective(x):
            w0, w1, w2, w3, w4, w5, w6 = x
            weighted_avg_pred_logits = [w0*x + w1*y + w2*z + w3*t + w4*a +w5*b +w6*c for x, y, z, t,a,b,c in zip(pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6)]
            return mean_squared_error(y_eval_cv,weighted_avg_pred_logits,squared=False)


        r_min, r_max = 0, 1
        bounds = np.array([[r_min, r_max], [r_min, r_max], [r_min, r_max], [r_min, r_max],[r_min, r_max],[r_min, r_max],[r_min, r_max]])
        result = dual_annealing(objective, bounds)
        solution = result['x']

        for i,col in enumerate(X_eval_cv):
            X_eval_cv[col] = X_eval_cv[col].multiply(solution[i])

        pred = X_eval_cv.apply(np.sum,axis=1)
        trues.extend(y_eval_cv)
        preds.extend(pred)


    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

IndexError: ignored

###Ridge Ensmeble

In [ ]:
drop_columns = ['content','wording','fold']
rmses = []

for target in ['content','wording']:
    preds = []
    trues = []

    for fold in range(4):
        ridge_train_cv = all_preds[all_preds["fold"] != fold].drop(columns=drop_columns)
        ridge_y_cv = all_preds[all_preds["fold"] != fold][target]
        ridge_train_cv = ridge_train_cv.filter(regex=(f'_{target}'))

        X_eval_cv = all_preds[all_preds["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = all_preds[all_preds["fold"] == fold][target]
        X_eval_cv = X_eval_cv.filter(regex=(f'_{target}'))

        ridge = Ridge(random_state=42)
        ridge.fit(ridge_train_cv,ridge_y_cv)

        for i,col in enumerate(X_eval_cv):
            X_eval_cv[col] = X_eval_cv[col].multiply(ridge.coef_[i])

        pred = X_eval_cv.apply(np.sum,axis=1)

        trues.extend(y_eval_cv)
        preds.extend(pred)


    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.46072068009387385
wording_rmse : 0.6428043430655724
mcrmse : 0.5517625115797231


####LGBM

In [ ]:
targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text",
                "prompt_question", "prompt_title",
                "prompt_text","prompt_length",
                "avg_word_length","semicolon_count","neg","neu","pos","compound",
                "exclamation_count","question_count","punctuation_sum","neg_prompt","neu_prompt","pos_prompt",
                "compound_prompt","flesch_reading_ease","flesch_kincaid_grade","gunning_fog","automated_readability_index",
                "coleman_liau_index","linsear_write_formula","dale_chall_readability_score","text_standard","spache_readability",
                "mcalpine_eflaw"
               ] + targets


model_dict = {}

for target in targets:
    models = []

    for fold in range(4):

        X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns)
        y_train_cv = train[train["fold"] != fold][target]

        mean_pred = all_preds[all_preds['fold']!=fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_{target}'))

        ridge = Ridge(random_state=42)
        ridge.fit(mean_pred,all_preds[all_preds['fold']!=fold][target])

        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(ridge.coef_[i])

        X_train_cv['ridge_pred'] =  mean_pred.apply(np.sum,axis=1)

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        mean_pred = all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_{target}'))

        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(ridge.coef_[i])

        X_eval_cv['ridge_pred'] =  mean_pred.apply(np.sum,axis=1)


        dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
        dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)

        params = {
                  'boosting_type': 'gbdt',
                  'random_state': 42,
                  'objective': 'regression',
                  'metric': 'rmse',
                  'learning_rate': 0.048,
                  'lambda_l1': 0.0,
                  'lambda_l2': 0.011
                  }

        evaluation_results = {}
        model = lgb.train(params,
                          num_boost_round=10000,
                            #categorical_feature = categorical_features,
                          valid_names=['train', 'valid'],
                          train_set=dtrain,
                          valid_sets=dval,
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=30, verbose=True),
                               lgb.log_evaluation(100),
                              lgb.callback.record_evaluation(evaluation_results)
                            ],
                          )
        models.append(model)

    model_dict[target] = models

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001037 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2749
[LightGBM] [Info] Number of data points in the train set: 5108, number of used features: 20
[LightGBM] [Info] Start training from score 0.017606
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[54]	train's rmse: 0.39945
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001191 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2671
[LightGBM] [Info] Number of data points in the train set: 5156, number of used features: 20
[LightGBM] [Info] Start training from score -0.039959
Training until validation scores don't improve for 30 rounds
[100]	train's rmse: 0.496807
[200]	train's rmse: 0.49598
Early stopping, best iteration is:
[175]	train's rmse: 0.494856
[Light

In [ ]:
# cv
rmses = []

for target in targets:
    models = model_dict[target]

    preds = []
    trues = []

    for fold, model in enumerate(models):

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        mean_pred = all_preds[all_preds['fold']!=fold].drop(columns=['content','wording','fold'])
        ridge = Ridge(random_state=42)
        ridge.fit(mean_pred,all_preds[all_preds['fold']!=fold][target])

        mean_pred= all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])

        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(ridge.coef_[i])

        X_eval_cv['ridge_pred'] =  mean_pred.apply(np.sum,axis=1)


        pred = model.predict(X_eval_cv)

        trues.extend(y_eval_cv)
        preds.extend(pred)

    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.44540256880891765
wording_rmse : 0.5894428054108428
mcrmse : 0.5174226871098803


###SVR

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
drop_columns = ['content','wording','fold']
rmses = []

for target in ['content','wording']:
    preds = []
    trues = []

    for fold in range(4):
        ridge_train_cv = all_preds[all_preds["fold"] != fold].drop(columns=drop_columns)
        ridge_y_cv = all_preds[all_preds["fold"] != fold][target]
        ridge_train_cv = ridge_train_cv.filter(regex=(f'_{target}'))

        X_eval_cv = all_preds[all_preds["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = all_preds[all_preds["fold"] == fold][target]
        X_eval_cv = X_eval_cv.filter(regex=(f'_{target}'))

        ridge = make_pipeline(StandardScaler(), SVR(kernel='linear',C=1.0, epsilon=0.2))
        ridge.fit(ridge_train_cv,ridge_y_cv)

        for i,col in enumerate(X_eval_cv):
            X_eval_cv[col] = X_eval_cv[col].multiply(ridge.named_steps['svr'].coef_[0][i])

        pred = X_eval_cv.apply(np.sum,axis=1)

        trues.extend(y_eval_cv)
        preds.extend(pred)


    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.4657488214635456
wording_rmse : 0.6467956025170459
mcrmse : 0.5562722119902958


In [ ]:
targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text",
                "prompt_question", "prompt_title",
                "prompt_text","prompt_length",
                "avg_word_length","semicolon_count","neg","neu","pos","compound",
                "exclamation_count","question_count","punctuation_sum","neg_prompt","neu_prompt","pos_prompt",
                "compound_prompt","flesch_reading_ease","flesch_kincaid_grade","gunning_fog","automated_readability_index",
                "coleman_liau_index","linsear_write_formula","dale_chall_readability_score","text_standard","spache_readability",
                "mcalpine_eflaw"
               ] + targets


model_dict = {}

for target in targets:
    models = []

    for fold in range(4):

        X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns)
        y_train_cv = train[train["fold"] != fold][target]

        mean_pred = all_preds[all_preds['fold']!=fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_{target}'))

        ridge = make_pipeline(StandardScaler(), SVR(kernel='linear',C=1.0, epsilon=0.2))
        ridge.fit(mean_pred,all_preds[all_preds['fold']!=fold][target])

        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(ridge.named_steps['svr'].coef_[0][i])

        X_train_cv['ridge_pred'] =  mean_pred.apply(np.sum,axis=1)
        # mean_pred = all_preds[all_preds['fold']!=fold].drop(columns=['content','wording','fold'])
        # mean_pred = mean_pred.filter(regex=(f'_wording'))
        # mean_pred = mean_pred.apply(np.mean,axis=1)
        # X_train_cv['mean_pred_wording'] = mean_pred

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        mean_pred = all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])
        mean_pred = mean_pred.filter(regex=(f'_{target}'))

        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(ridge.named_steps['svr'].coef_[0][i])

        X_eval_cv['ridge_pred'] =  mean_pred.apply(np.sum,axis=1)

        # mean_pred = all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])
        # mean_pred = mean_pred.filter(regex=(f'_wording'))
        # mean_pred = mean_pred.apply(np.mean,axis=1)
        # X_eval_cv['mean_pred_wording'] = mean_pred


        dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
        dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)

        params = {
                  'boosting_type': 'gbdt',
                  'random_state': 42,
                  'objective': 'regression',
                  'metric': 'rmse',
                  'learning_rate': 0.048,
                  'lambda_l1': 0.0,
                  'lambda_l2': 0.011
                  }

        evaluation_results = {}
        model = lgb.train(params,
                          num_boost_round=10000,
                            #categorical_feature = categorical_features,
                          valid_names=['train', 'valid'],
                          train_set=dtrain,
                          valid_sets=dval,
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=30, verbose=True),
                               lgb.log_evaluation(100),
                              lgb.callback.record_evaluation(evaluation_results)
                            ],
                          )
        models.append(model)

    model_dict[target] = models

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001284 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2749
[LightGBM] [Info] Number of data points in the train set: 5108, number of used features: 20
[LightGBM] [Info] Start training from score 0.017606
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[65]	train's rmse: 0.394069
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001300 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2671
[LightGBM] [Info] Number of data points in the train set: 5156, number of used features: 20
[LightGBM] [Info] Start training from score -0.039959
Training until validation scores don't improve for 30 rounds
[100]	train's rmse: 0.493765
Early stopping, best iteration is:
[123]	train's rmse: 0.49356
[LightGBM] [Warning] Auto-choosing

In [ ]:
rmses = []

for target in targets:
    models = model_dict[target]

    preds = []
    trues = []

    for fold, model in enumerate(models):

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        mean_pred = all_preds[all_preds['fold']!=fold].drop(columns=['content','wording','fold'])
        ridge = make_pipeline(StandardScaler(), SVR(kernel='linear',C=1.0, epsilon=0.2))
        ridge.fit(mean_pred,all_preds[all_preds['fold']!=fold][target])

        mean_pred= all_preds[all_preds['fold']==fold].drop(columns=['content','wording','fold'])

        for i,col in enumerate(mean_pred):
            mean_pred[col] = mean_pred[col].multiply(ridge.named_steps['svr'].coef_[0][i])

        X_eval_cv['ridge_pred'] =  mean_pred.apply(np.sum,axis=1)


        pred = model.predict(X_eval_cv)

        trues.extend(y_eval_cv)
        preds.extend(pred)

    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.4421259267987118
wording_rmse : 0.5955446103201631
mcrmse : 0.5188352685594375


###LGMB - Predictions as Features

In [ ]:
targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text",
                "prompt_question", "prompt_title",
                "prompt_text","prompt_length",
                "avg_word_length","semicolon_count","neg","neu","pos","compound",
                "exclamation_count","question_count","punctuation_sum","neg_prompt","neu_prompt","pos_prompt",
                "compound_prompt","flesch_reading_ease","flesch_kincaid_grade","gunning_fog","automated_readability_index",
                "coleman_liau_index","linsear_write_formula","dale_chall_readability_score","text_standard","spache_readability",
                "mcalpine_eflaw"
               ] + targets




model_dict = {}

for target in targets:
    models = []

    for fold in range(4):

        X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns)
        y_train_cv = train[train["fold"] != fold][target]

        features_pred = all_preds[all_preds["fold"] != fold].drop(columns=['content','wording','fold'])
        X_train_cv = pd.concat([X_train_cv,features_pred],axis=1)

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        features_pred = all_preds[all_preds["fold"] == fold].drop(columns=['content','wording','fold'])
        X_eval_cv = pd.concat([X_eval_cv,features_pred],axis=1)

        dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
        dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)

        params = {
                  'boosting_type': 'gbdt',
                  'random_state': 42,
                  'objective': 'regression',
                  'metric': 'rmse',
                  'learning_rate': 0.048,
                  'lambda_l1': 0.0,
                  'lambda_l2': 0.011
                  }

        evaluation_results = {}
        model = lgb.train(params,
                          num_boost_round=10000,
                            #categorical_feature = categorical_features,
                          valid_names=['train', 'valid'],
                          train_set=dtrain,
                          valid_sets=dval,
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=30, verbose=True),
                               lgb.log_evaluation(100),
                              lgb.callback.record_evaluation(evaluation_results)
                            ],
                          )
        models.append(model)

    model_dict[target] = models

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000797 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3570
[LightGBM] [Info] Number of data points in the train set: 5108, number of used features: 14
[LightGBM] [Info] Start training from score 0.017606
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[48]	train's rmse: 0.420033
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000839 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3570
[LightGBM] [Info] Number of data points in the train set: 5156, number of used features: 14
[LightGBM] [Info] Start training from score -0.039959
Training until validation scores don't improve for 30 rounds
[100]	train's rmse: 0.547879
Early stopping, best iteration is:
[78]	train's rmse: 0.547834
[LightGBM] [Warning] Auto-choosing

In [ ]:
# cv
rmses = []

for target in targets:
    models = model_dict[target]

    preds = []
    trues = []

    for fold, model in enumerate(models):

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]
        features_pred = all_preds[all_preds["fold"] == fold].drop(columns=['content','wording','fold'])
        X_eval_cv = pd.concat([X_eval_cv,features_pred],axis=1)

        pred = model.predict(X_eval_cv)

        trues.extend(y_eval_cv)
        preds.extend(pred)

    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.47584624950360127
wording_rmse : 0.6187444225296523
mcrmse : 0.5472953360166268
